In [1]:
import os
import netCDF4
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature

In [2]:
## Load Data
#Path
os.chdir('D:\\Bigfiles\\ERA5_PCP')
filename_PCP_accum1 = 'ERA5_Accum.nc'

#Load netcdf
ncdf1 = netCDF4.Dataset(filename_PCP_accum1)
print(ncdf1)

#Get dimension
lons = ncdf1.variables['lon'][:]
lats = ncdf1.variables['lat'][:]
times = ncdf1.variables['time'][:]

#Get variables
pcp = ncdf1.variables['pcp_accum'][:]

#Calculate datetime
dtimes = np.empty(len(times), dtype='datetime64[h]')
for i,t in enumerate(times):
    dtimes[i] = np.datetime64('1900-01-01') + np.timedelta64(np.int64(t),'h')
dtimes = dtimes.astype('datetime64')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): lon(121), lat(1440), time(14973)
    variables(dimensions): float32 lon(lon), float32 lat(lat), float64 time(time), float32 pcp_accum(time,lon,lat)
    groups: 


In [3]:
#Print Variables
print(ncdf1.variables.keys())
print(ncdf1.dimensions)
print(ncdf1.variables['pcp_accum'])

print(ncdf1.variables['pcp_accum'][:,1,:])
print(ncdf1.variables['pcp_accum'][1,:,:])
print(ncdf1.variables['pcp_accum'][:,:,1])

#print(ncdf3.variables['pcp_accum'][14091,239,240])

#print(ncdf1.variables['ilon'][:])
#print(ncdf1.variables['ilat'][:])
#print(ncdf1.variables['itime'][:])


dict_keys(['lon', 'lat', 'time', 'pcp_accum'])
{'lon': <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 121, 'lat': <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 1440, 'time': <class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 14973}
<class 'netCDF4._netCDF4.Variable'>
float32 pcp_accum(time, lon, lat)
    units: m
unlimited dimensions: 
current shape = (14973, 121, 1440)
filling on, default _FillValue of 9.969209968386869e+36 used


<ipython-input-3-61b15370aedb>:4: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  print(ncdf1.variables['pcp_accum'])


[[1.0382205e-02 8.3689271e-03 6.0494076e-03 ... 6.9511845e-03
  9.7241802e-03 1.2005663e-02]
 [1.1145417e-02 9.0455832e-03 6.6149263e-03 ... 8.2255751e-03
  1.0867509e-02 1.2895019e-02]
 [1.1152274e-02 9.0538636e-03 6.6247592e-03 ... 8.2369605e-03
  1.0876695e-02 1.2901747e-02]
 ...
 [4.4058506e+01 4.3760731e+01 4.3543755e+01 ... 4.4783092e+01
  4.4548744e+01 4.4329758e+01]
 [4.4067436e+01 4.3770256e+01 4.3554367e+01 ... 4.4790295e+01
  4.4556484e+01 4.4338078e+01]
 [4.4067436e+01 4.3770256e+01 4.3554367e+01 ... 4.4790295e+01
  4.4556484e+01 4.4338078e+01]]
[[1.0441850e-02 1.0371078e-02 8.1576500e-03 ... 8.9193089e-03
  9.7435871e-03 1.0509904e-02]
 [1.1145417e-02 9.0455832e-03 6.6149263e-03 ... 8.2255751e-03
  1.0867509e-02 1.2895019e-02]
 [1.1268198e-02 8.1899958e-03 4.8302626e-03 ... 9.9350689e-03
  1.2464184e-02 1.2113694e-02]
 ...
 [3.5061905e-05 3.5061905e-05 3.4544388e-05 ... 3.6096942e-05
  3.6096942e-05 3.5061905e-05]
 [5.2140029e-05 5.2140029e-05 5.2140029e-05 ... 5.2140029e-

In [4]:
print(ncdf1.variables['pcp_accum'][:,1,1])

[8.3689271e-03 9.0455832e-03 9.0538636e-03 ... 4.3760731e+01 4.3770256e+01
 4.3770256e+01]


In [5]:
#Obtain
roll_days = [1,2,3,4,5,6,7,8,9,10,15,20,30,60,90,180,270,365,545,730]
rolling_max = np.zeros((np.shape(pcp)[1], np.shape(pcp)[2]))
rolling_mean = np.zeros((np.shape(pcp)[1], np.shape(pcp)[2]))
#rolling_p95 = np.zeros((np.shape(pcp)[1], np.shape(pcp)[2]))
#rolling_p75 = np.zeros((np.shape(pcp)[1], np.shape(pcp)[2]))
#rolling_p50 = np.zeros((np.shape(pcp)[1], np.shape(pcp)[2]))
#rolling_p25 = np.zeros((np.shape(pcp)[1], np.shape(pcp)[2]))
#rolling_p05 = np.zeros((np.shape(pcp)[1], np.shape(pcp)[2]))

roll_days = [60,90,180,270,365,545,730]
for n in roll_days:
    print(n)
    PCP_n = pcp[n:,:,:] - pcp[:-n,:,:]
    rolling_max[:,:] = PCP_n.max(axis=0)

    #Save max for each lon and lat
    np.savetxt('ERA5_PCP_max_' + str(n) + 'd.csv', rolling_max*1000, delimiter=';')

    ##Properties of max for this duration
    #lon, lat, height
    Ant_max = np.max(rolling_max)
    Ant_max_coords = np.where(rolling_max == Ant_max)
    print(Ant_max)
    print(lats[Ant_max_coords[1]])
    print(lons[Ant_max_coords[0]])

    #time
    Ant_max_timecoords = np.where(PCP_n == Ant_max)
    print(dtimes[Ant_max_timecoords[0]])

print('end')

60
1.2562103271484375
[-64.75]
[-66.]
['2010-09-27T00']
90
1.64947509765625
[-64.75]
[-66.]
['2001-08-03T00']
180
2.7162036895751953
[-64.75]
[-66.]
['1989-05-04T00']
270
3.5699920654296875
[-64.75]
[-66.]
['2010-03-02T00']
365
4.266571044921875
[-64.75]
[-66.]
['2001-02-25T00']
545
5.936553955078125
[-64.75]
[-66.]
['1998-12-26T00']
730
7.812534332275391
[-64.75]
[-66.]
['1998-03-12T00']
end


In [32]:
np.savetxt('PCP_max_lats.csv', lons, delimiter=';')
np.savetxt('PCP_max_lons.csv', lats, delimiter=';')
roll_days = [1,2,3,4,5,6,7,8,9,10,15,20,30,60,90,180,270,365,545,730]
np.savetxt('PCP_max_duration.csv', roll_days, delimiter=';')